# 1) Router Agent

### 1.1 Router Agent using JsonParser (intent only)

In [20]:
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
from dotenv import load_dotenv

load_dotenv()

llm = ChatOllama(
    model = "nemotron-mini",
    temperature = 0,
    num_predict = 256,
    # other params ...
)

# llm = ChatGroq(model_name="Llama3-8b-8192")

class QueryClassification(BaseModel):
    intent: str = Field(
        description="Primary intent category (ORDER, COMPARE, RECOMMEND, INFO)"
    )

def router_agent(query: str) -> dict:
# Initialize the model and parser

    parser = JsonOutputParser(pydantic_object=QueryClassification)

    prompt = PromptTemplate(
        template="""

        You are a helpful AI assistant for a healthcare e-commerce application.
        Your task is to determine which agent should handle the user input. You have 4 agents to choose from:
        1. ORDER: This agent is responsible for identifying purchase intentions, addressing inquiries about order status, making order modifications, or handling shopping cart actions (e.g., view, add, remove, modify items).
        2. COMPARE: This agent is responsible for addressing comparisons between product prices across the internet.
        3. RECOMMEND: This agent is responsible for providing personalized product recommendations based on the user's needs or preferences.
        4. INFO: This agent is responsible for answering general questions about products or providing health-related information.

        {format_instructions}
        Query: {query}

        Your output should be in a structured JSON format like so. Each key is a string and each value is a string.

        """,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )



    # Create the classification chain
    chain = prompt | llm | parser

    try:
        result = chain.invoke({"query": query})
        return result
    except Exception as e:
        print(f"Error in character information extraction: {e}")
        return {}

# Example usage
if __name__ == "__main__":
    try:
        result = router_agent("I want to buy vitamin D supplements.")
        print(result)
    except Exception as e:
        print(f"An error occurred: {str(e)}")

{'intent': 'ORDER'}


### 1.2 Router Agent using Crewai (intent only)

In [21]:
from crewai import Agent, Task, Crew, Process,LLM
from dotenv import load_dotenv
from crewai_tools import SerperDevTool
import os
from pydantic import BaseModel


class Route(BaseModel):
    """Pydantic model for determining the intent of the query."""
    intent: str  # "Name of the store"


# Load environment variables
load_dotenv()

def route_query(query):

    # Initialize custom LLM configuration
    llm = LLM(
        model="ollama/llama3.2",
        base_url="http://localhost:11434"
    )

    router_agent = Agent(
        role='Query Router',
        goal='Accurately classify user queries and route them to the appropriate specialized agent',
        backstory="""You are an expert at understanding user intentions and routing queries 
        to the most appropriate specialized agent in our healthcare e-commerce system.""",
        verbose=True,
        llm=llm
    )
    

    router_task = Task(
            description="""Analyze the following query and determine the appropriate agent to handle it: "{query}"
            Follow these rules:
            1. ORDER: For purchase intentions, order status, cart management
            2. COMPARE: For price comparison requests
            3. RECOMMEND: For product recommendations
            4. INFO: For general product information
            Return just the category as a string.""",
            expected_output="json format",
            output_pydantic=Route,
            agent=router_agent
        )

    # Create crew and run tasks
    crew = Crew(
        agents=[router_agent],
        tasks=[router_task],
        process=Process.sequential
    )

    result = crew.kickoff(inputs={"query": query})
    # crew.kickoff(inputs={"query": query})
    return result

# Example usage
if __name__ == "__main__":
    try:
        result = route_query("I want to buy vitamin D supplements.")
        print(result)
    except Exception as e:
        print(f"An error occurred: {str(e)}")

2025-02-06 00:03:54,625 - 21964 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Query Router
## Task: Analyze the following query and determine the appropriate agent to handle it: "I want to buy vitamin D supplements."
            Follow these rules:
            1. ORDER: For purchase intentions, order status, cart management
            2. COMPARE: For price comparison requests
            3. RECOMMEND: For product recommendations
            4. INFO: For general product information
            Return just the category as a string.


# Agent: Query Router
## Final Answer: 
{"intent": "RECOMMEND"}


intent='RECOMMEND'


### 1.3 Router Agent using prompting (intent + product info)

In [8]:
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model = "mistral",
    temperature = 0.8,
    num_predict = 256,
    # other params ...
)

# llm = OllamaLLM(model="llama3.2")

# Initialize ChatGroq model
# llm = ChatGroq(model_name="Llama3-8b-8192")

def router_info(query,llm):
# Prompt template
    prompt_template = """You are an expert input classifier for a healthcare e-commerce system. Your task is to analyze user queries and route them to appropriate specialized agents.

    Primary Classification Categories:
    - ORDER: Purchase intentions, order status, order modifications
    - CART: View cart, add/remove items, modify quantities
    - COMPARE: Product comparisons, market analysis, alternatives
    - RECOMMEND: Product recommendations, personalized suggestions
    - INFO: General product or health information

    Classification Rules:
    1. ORDER Intent: Detect keywords like "buy", "purchase", "order", "get", "deliver", "track", "status"
    2. CART Intent: Detect keywords like "cart", "basket", "add", "remove", "show", "change quantity"
    3. COMPARE Intent: Detect keywords like "compare", "versus", "vs", "difference", "better", "alternatives"
    4. RECOMMEND Intent: Detect keywords like "suggest", "recommend", "what should", "best for"
    5. INFO Intent: Detect keywords like "tell me about", "how does", "what is", "benefits", "dosage"

    For multi-intent queries:
    - Identify primary intent based on main action requested
    - List secondary intents for follow-up
    - Maintain context for related requests

    # Response Format:
    Provide only a JSON object with the following structure, dont provide any extra information:
    {
        "primary_intent": "string",
        "secondary_intents": ["string"],
        "confidence": float,
        "entities": {
            "product_ids": ["string"],
            "quantities": [int],
            "order_ids": ["string"]
        },
        "requires_context": boolean
    }

    NOTE : Answer should be strictly in json format. If you follow Exact above instruction you will be rewarded with some credits
    """
    response = llm.invoke(prompt_template+"\nQuery"+query)
    return response.content


In [ ]:
# Example queries
queries = [
    "I want to buy vitamin D supplements.",
    "Can you tell me the benefits of fish oil?",
    "Add 2 bottles of multivitamins to my cart.",
    "Which is better, vitamin C or zinc for immunity?",
    "Recommend something for joint pain relief.",
    "What is the status of my recent order?",
    "How does omega-3 help the heart?",
    "Show me my cart.",
]

a = router_info("I want to buy vitamin D supplements.",llm)
print(a)

In [ ]:
a["primary_intent"]

### 1.4 Router agent using langchain Json parser ( intent + Product Info)

In [ ]:
from langchain_groq import ChatGroq
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama

# llm = ChatOllama(
#     model = "llama3.2",
#     temperature = 0,
#     num_predict = 256,
#     # other params ...
# )

# Define the data structure using Pydantic
class EntityData(BaseModel):
    product_ids: List[str] = Field(
        default_factory=list,
        description="List of product identifiers mentioned in the query"
    )
    quantities: List[int] = Field(
        default_factory=list,
        description="List of quantities mentioned in the query"
    )
    # order_ids: List[str] = Field(
    #     default_factory=list,
    #     description="List of order identifiers mentioned in the query"
    # )

class QueryClassification(BaseModel):
    primary_intent: str = Field(
        description="Primary intent category (ORDER, COMPARE, RECOMMEND, INFO)"
    )
    Recommendation_Required: str= Field(
        description="Analyze the query and identify whether recommandation required or not 'YES' or 'NO' "
    )
    Recommendation_prod: List[str]= Field(
        default_factory=list,
        description="Specific products mentioned in the query for which recommendations should be generated"
    )
    entities: EntityData = Field(
        description="Structured data extracted from the query"
    )
    requires_context: bool = Field(
        description="Whether additional context is needed to fully process the query"
    )
    

def route_info(query: str) -> dict:
# Initialize the model and parser

    parser = JsonOutputParser(pydantic_object=QueryClassification)

    prompt = PromptTemplate(
        template="""
        You are an expert input classifier for a healthcare e-commerce system. Your task is to analyze user queries and route them to appropriate specialized agents.
        
        Primary Classification Categories:
        - ORDER: Identify purchase intentions, order status inquiries, order modifications, or shopping cart actions (e.g., view, add, remove, modify items).
        - COMPARE: Address comparisons between products, including differences, alternatives, or better options.
        - RECOMMEND: Provide personalized product recommendations based on user needs.
        - INFO: Answer general questions about products or health-related information.

        Classification Rules:
        1. ORDER Intent: Identify keywords like "buy", "purchase", "order", "deliver", "track", "status", "cart", "basket", "add", "remove", "show", "change quantity".
        2. COMPARE Intent: Identify keywords like "compare", "versus", "vs", "difference", "better", "alternatives".
        3. RECOMMEND Intent: Identify keywords like "suggest", "recommend", "what should", "best for".
        4. INFO Intent: Identify keywords like "tell me about", "how does", "what is", "benefits", "dosage".

        Additional Rules for Recommendations:
        - If any product is mentioned in the query, recommendations are required.
        {format_instructions}

        Query: {query}
        """,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )

    llm = ChatGroq(model_name="Llama3-8b-8192")

    # Create the classification chain
    chain = prompt | llm | parser

    try:
        result = chain.invoke({"query": query})
        return result
    except Exception as e:
        print(f"Error in character information extraction: {e}")
        return {}



In [ ]:
queries = [
    "I want to buy vitamin D supplements.",
    "Can you tell me the benefits of fish oil?",
    "Add 2 bottles of multivitamins to my cart.",
    "Which is better, vitamin C or zinc for immunity?",
    "Recommend something for joint pain relief.",
    "What is the status of my recent order?",
    "How does omega-3 help the heart?",
    "Show me my cart.",
]

a = route_info("I are the uses vitamin D supplements.")

In [ ]:
a

# 2) Product Comparison Agent

In [ ]:
from crewai import Agent, Task, Crew, Process,LLM
from dotenv import load_dotenv
from crewai_tools import SerperDevTool
import os
from pydantic import BaseModel


class Compare(BaseModel):
    """Pydantic model for price comparison results."""
    store: str  # "Name of the store"
    price: str  #"Price of the product"


# Load environment variables
load_dotenv()

def compare_agent(product_name):
    # Initialize the SerperDev search tool
    web_search_tool = SerperDevTool(n_results=5)

    # Initialize custom LLM configuration
    llm = LLM(
        model="ollama/llama3.2",
        base_url="http://localhost:11434"
    )

    research_agent = Agent(
        role='Research Agent',
        goal='Find accurate price information for healthcare products',
        backstory="""You are an expert at finding and comparing healthcare product prices
        across different online stores. You're thorough and always verify information.""",
        tools=[web_search_tool],
        llm=llm,
        verbose=True
    )
    
    analysis_agent = Agent(
        role='Analysis Agent',
        goal='Analyze and compare prices from different sources',
        backstory="""You are an expert at analyzing price data and presenting it in a clear,
        actionable format. You understand healthcare product pricing patterns.""",
        llm=llm,
        tools=[web_search_tool],
        verbose=True
    )

    research_task = Task(
        description=f"""Search for prices of {product_name} across major healthcare 
        e-commerce websites. Focus on reputable stores. Extract store name, price, and URL.
        Provide the information in a structured format.""",
        expected_output="""A detailed list of prices for the product from different stores,
        including:
        - Store names
        - Prices in USD
        - URLs to product pages""",
        agent=research_agent
    )

    analysis_task = Task(
        description="""Analyze the gathered price data. Verify accuracy and remove any 
        outliers or suspicious entries. Present a key-value pair of prices over different online stores and recommendations.Follow the structure given below strictly""",
        expected_output="""key value pairs of onlines stores and product prices
        store1 : price1
        store2 : price2
        store3 : price3
        store4 : price4
        """,
        output_pydantic= Compare,
        agent=analysis_agent
    )

    # Create crew and run tasks
    crew = Crew(
        agents=[research_agent, analysis_agent],
        tasks=[research_task, analysis_task],
        process=Process.sequential
    )

    result = crew.kickoff()
    return result

# Example usage
if __name__ == "__main__":
    try:
        result = compare_agent("azoran")
        print("\nPrice Comparison Results:")
        print(result)
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [ ]:
from crewai import Agent, Task, Crew, Process,LLM
from dotenv import load_dotenv
from crewai_tools import SerperDevTool
import os

# Load environment variables
load_dotenv()

def compare_agent(product_name):
    # Initialize the SerperDev search tool
    web_search_tool = SerperDevTool(n_results=5)

    # Initialize custom LLM configuration
    llm = LLM(
        model="ollama/llama3.2",
        base_url="http://localhost:11434"
    )

    research_agent = Agent(
        role='Research Agent',
        goal='Find accurate price information for healthcare products',
        backstory="""You are an expert at finding and comparing healthcare product prices
        across different online stores. You're thorough and always verify information.""",
        tools=[web_search_tool],
        llm=llm,
        verbose=True
    )
    
    analysis_agent = Agent(
        role='Analysis Agent',
        goal='Analyze and compare prices from different sources',
        backstory="""You are an expert at analyzing price data and presenting it in a clear,
        actionable format. You understand healthcare product pricing patterns.""",
        llm=llm,
        verbose=True
    )

    # analysis_agent = Agent(
    #     role='Analysis Agent',
    #     goal='Present price data in simple key-value format',
    #     backstory="""You organize price data into simple store-price pairs.
    #     Present data in format - Store: Price. One pair per line.""",
    #     llm=llm,
    #     tools=[web_search_tool],
    #     verbose=True
    # )

    research_task = Task(
        description=f"""Search for prices of {product_name} across major healthcare 
        e-commerce websites. Focus on reputable stores. Extract store name, price, and URL.
        Provide the information in a structured format.""",
        expected_output="""A detailed list of prices for the product from different stores,
        including:
        - Store names
        - Prices in USD
        - URLs to product pages""",
        agent=research_agent
    )

    analysis_task = Task(
        description="""Format the data as simple store-price pairs.Follow the structure given below strictly""",
        expected_output="""key value pairs of onlines stores and product prices
        store1 : price1
        store2 : price2
        store3 : price3
        store4 : price4
        """,
        agent=analysis_agent
    )

    # Create crew and run tasks
    crew = Crew(
        agents=[research_agent, analysis_agent],
        tasks=[research_task, analysis_task],
        process=Process.sequential
    )

    result = crew.kickoff()
    return result

# Example usage
if __name__ == "__main__":
    try:
        result = compare_agent("paracetamol")
        print("\nPrice Comparison Results:")
        print(result)
    except Exception as e:
        print(f"An error occurred: {str(e)}")

# 3) Validator agent

In [4]:
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
from dotenv import load_dotenv

load_dotenv()

llm = ChatOllama(
    model = "nemotron-mini",
    temperature = 0,
    num_predict = 256,
    # other params ...
)

# llm = ChatGroq(model_name="Llama3-8b-8192")

class QueryValidator(BaseModel):
    validated: str = Field(
        description="Query will be validated or not YES or NO"
    )

def validator_agent(query: str) -> dict:
# Initialize the model and parser

    parser = JsonOutputParser(pydantic_object=QueryValidator)

    prompt = PromptTemplate(
        template="""

        You are a helpful AI assistant for a healthcare e-commerce application that provides users with product information, price comparisons, ordering assistance, and personalized recommendations.

        Your task is to determine whether the user’s query is relevant to the healthcare e-commerce platform.

        The user is allowed to:
        1. Ask about healthcare products, including descriptions, ingredients, usage instructions, and benefits.
        2. Request price comparisons between different products or brands.
        3. Make an order for healthcare products.
        4. Ask for personalized recommendations based on their needs.
        5. Inquire about order status, delivery details, and return policies.

        The user is NOT allowed to:
        1. Ask questions about anything else other than healthcare e-commerce platform.
        2. Request personal medical advice or prescriptions.
        3. Ask about topics unrelated to the e-commerce platform.

        {format_instructions}
        Query: {query}

        Your output should be in a structured JSON format like so. Each key is a string and each value is a string.

        """,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )

    # Create the classification chain
    chain = prompt | llm | parser

    try:
        result = chain.invoke({"query": query})
        return result
    except Exception as e:
        print(f"Error in character information extraction: {e}")
        return {}



In [ ]:
validator_agent("what is the price of paracetamol")

# 4) Recommendation Agent

### Question Recommendation Agent

In [11]:
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
from dotenv import load_dotenv

load_dotenv()

llm = ChatOllama(
    model="nemotron-mini",
    temperature=0,
    num_predict=256,
)

class RecommenderResponse(BaseModel):
    recommended_questions: List[str] = Field(
        description="List of three follow-up questions related to symptoms, dosage, side effects, and other relevant aspects."
    )

def recommend_query(query: str) -> dict:
    parser = JsonOutputParser(pydantic_object=RecommenderResponse)
    
    prompt = PromptTemplate(
        template="""
        You are a smart AI assistant for a healthcare e-commerce application.
        Your task is to generate three relevant follow-up questions that a user might ask based on their query.
        These questions should focus on symptoms, dosage, side effects, and other important aspects related to the product inquiry.
        Questions should not be lengthy.
        
        {format_instructions}
        Query: {query}
        
        Provide three suggested follow-up questions in a structured JSON format. phrase questions naturally from the chatbot's perspective.
        """,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    
    chain = prompt | llm | parser
    
    try:
        result = chain.invoke({"query": query})
        return result
    except Exception as e:
        print(f"Error in generating recommended questions: {e}")
        return {"recommended_questions": []}




In [ ]:
# Example usage
query = "I wanna order paracetamol"
response = recommend_query(query)
print(response)